# Lec. 18 Global Optimization (II)

Try different minimization methods in scipy on larger systems ($N$ up to 20), and show

1 the average number of attempts to find the ground state

2 the time costs

In [1]:
from scipy.optimize import minimize
import requests
import numpy as np
import timeit
from numba import jit

#Below are all of the needed functions

def get_pos_from_url(address='http://doye.chem.ox.ac.uk/jon/structures/LJ/points/', N=7):
    '''
    function to retrieve ground state data from website
    '''
    url_address = address + str(N)
    data_str = requests.get(url_address).text
    return parse_url_text(data_str)    
    
def parse_url_text(data_str):
    x_array = []
    text = data_str.split('\n')
    for line in text:
        [x_array.append(float(i)) for i in line.split()]
    return np.array(x_array)

@jit
def LJ(r, epsilon=1.0, sigma=1.0):
    '''
    function to calculate the Lennard-Jones potential
    '''
    r6 = r**6          #attractive
    r12 = r6**2        #repulsive
    sigma6 = sigma**6
    sigma12 = sigma6**2
    V = (4*epsilon*((sigma12/r12)-(sigma6/r6)))
    return V

@jit
def init_pos(N, L=5):
    '''
    function to generate random initial position
    '''
    return L*np.random.random_sample((N*3))

@jit
def total_energy(positions):
    '''
    function to calculate total energy
    '''
    E = 0
    N_atom = int(len(positions)/3)
    for i in range (N_atom-1):
        for j in range(i+1, N_atom):
            pos1 = positions[i*3:(i+1)*3]
            pos2 = positions[j*3:(j+1)*3]
            dist = np.linalg.norm(pos1-pos2)
            E += LJ(dist)
    return E


## For N=8 atoms

In [2]:
f1_N8_values = []
x1_N8_values = []
N_attempts = 20
N_atom = 8

start = timeit.default_timer()

print('CG minimization method\n')

for i in range(N_attempts):
    pos = init_pos(N_atom)
    res1N8 = minimize(total_energy, pos, method='CG', tol=1e-4)
    f1_N8_values.append(res1N8.fun)
    x1_N8_values.append(res1N8.x)
    print('step:', i, 'value:', res1N8.fun)
    
stop = timeit.default_timer()    

print('\nGlobal min:', min(f1_N8_values))

pos = get_pos_from_url(N=8)
print('Real ground state:', total_energy(pos))
print('\ntime elapsed:', stop-start, 'seconds')

print('found after average 5 steps')

CG minimization method

step: 0 value: -9.103852417826754
step: 1 value: -19.821489191943776
step: 2 value: -18.85682616748586
step: 3 value: -18.778208173775358
step: 4 value: -19.821489191922716
step: 5 value: -18.85682616693661
step: 6 value: -19.765297850586496
step: 7 value: -19.821489191847405
step: 8 value: -18.77820817372194
step: 9 value: -19.821489192127835
step: 10 value: -18.8568261675617
step: 11 value: -19.821489192069116
step: 12 value: -18.778208173154997
step: 13 value: -18.828671549478585
step: 14 value: -19.821489191936664
step: 15 value: -19.82148919152781
step: 16 value: -19.821489191836882
step: 17 value: -19.76529785072553
step: 18 value: -18.856826167405988
step: 19 value: -18.77820817392233

Global min: -19.821489192127835
Real ground state: -19.821489187804723

time elapsed: 3.4417733790005514 seconds
found after average 5 steps


In [3]:
f2_N8_values = []
x2_N8_values = []

start = timeit.default_timer()

print('Nelder-Mead minimization method\n')

for j in range(N_attempts+75):
    pos = init_pos(N_atom)
    res2N8 = minimize(total_energy, pos, method='nelder-mead', tol=1e-4)
    f2_N8_values.append(res2N8.fun)
    x2_N8_values.append(res2N8.x)
    print('step:', j, 'value:', res2N8.fun)
   
stop = timeit.default_timer()    

print('\nGlobal min:', min(f2_N8_values))

pos = get_pos_from_url(N=8)
print('Real ground state:', total_energy(pos))
print('\ntime elapsed:', stop-start, 'seconds')

print('This method does not get ground state even after many iterations')

Nelder-Mead minimization method

step: 0 value: -9.917516395818918
step: 1 value: -9.719884579590746
step: 2 value: -9.051849848300938
step: 3 value: -7.883542354377156
step: 4 value: -5.123992101399309
step: 5 value: -5.005850472707278
step: 6 value: -7.123776625588984
step: 7 value: -15.569795911970907
step: 8 value: -12.312533749300798
step: 9 value: -7.31395398140516
step: 10 value: -8.196946360249362
step: 11 value: -7.49129571195524
step: 12 value: -5.086703716878271
step: 13 value: -9.04479044003771
step: 14 value: -7.090118479401157
step: 15 value: -10.407836392643528
step: 16 value: -12.33670299959896
step: 17 value: -6.077993500141365
step: 18 value: -8.696041415610669
step: 19 value: -9.819734348290211
step: 20 value: -9.631880301575396
step: 21 value: -9.691206548505905
step: 22 value: -7.101467816621367
step: 23 value: -8.082832024098767
step: 24 value: -9.129380644251937
step: 25 value: -8.25491519466876
step: 26 value: -7.421303581224349
step: 27 value: -10.1407656911331

In [4]:
f3_N8_values = []
x3_N8_values = []

start = timeit.default_timer()

print('BFGS minimization method\n')

for k in range(N_attempts):
    pos = init_pos(N_atom)
    res3N8 = minimize(total_energy, pos, method='BFGS', tol=1e-4)
    f3_N8_values.append(res3N8.fun)
    x3_N8_values.append(res3N8.x)
    print('step:', k, 'value:', res3N8.fun)
    
stop = timeit.default_timer()    

print('\nGlobal min:', min(f3_N8_values))

pos = get_pos_from_url(N=8)
print('Real ground state:', total_energy(pos))
print('\ntime elapsed:', stop-start, 'seconds')

print('found after average 2 steps')

BFGS minimization method

step: 0 value: -19.821489192087984
step: 1 value: -19.82148919212204
step: 2 value: -18.856826167765377
step: 3 value: -19.821489192075067
step: 4 value: -18.85682616766367
step: 5 value: -19.821489192106558
step: 6 value: -19.1692797711512
step: 7 value: -19.821489192132017
step: 8 value: -18.856826167727128
step: 9 value: -19.82148919213637
step: 10 value: -18.828671549566373
step: 11 value: -19.765297850843833
step: 12 value: -18.856826167741424
step: 13 value: -19.821489192031425
step: 14 value: -18.856826167757653
step: 15 value: -9.103854163148569
step: 16 value: -19.82148919211359
step: 17 value: -18.82867154955965
step: 18 value: -15.533064235359912
step: 19 value: -18.856826167722

Global min: -19.82148919213637
Real ground state: -19.821489187804723

time elapsed: 1.5887560270002723 seconds
found after average 2 steps


In [5]:
f4_N8_values = []
x4_N8_values = []

start = timeit.default_timer()

print('Powell minimization method\n')

for m in range(N_attempts+75):
    pos = init_pos(N_atom)
    res4N8 = minimize(total_energy, pos, method='powell', tol=1e-4)
    f4_N8_values.append(res4N8.fun)
    x4_N8_values.append(res4N8.x)
    print('step:', m, 'value:', res4N8.fun)
        
stop = timeit.default_timer()    

print('\nGlobal min:', min(f4_N8_values))

pos = get_pos_from_url(N=8)
print('Real ground state:', total_energy(pos))
print('\ntime elapsed:', stop-start, 'seconds')
print('This method does not produce the proper ground state')

Powell minimization method

step: 0 value: -19.820400079603232
step: 1 value: -18.828313655281992
step: 2 value: -12.003975597998036
step: 3 value: -19.82086924062682
step: 4 value: -18.856682745383576
step: 5 value: -16.751412542114867
step: 6 value: -19.821045866629536
step: 7 value: -18.778158724550142
step: 8 value: -12.110124314334115
step: 9 value: -19.820076220508444
step: 10 value: -18.77763921301977
step: 11 value: -12.025909958429494
step: 12 value: -18.778160792855136
step: 13 value: -12.149406817281523
step: 14 value: -12.075589728065458
step: 15 value: -19.821325104071747
step: 16 value: -17.834937422896598
step: 17 value: -18.775704905344362
step: 18 value: -18.77392278556778
step: 19 value: -12.168013505190546
step: 20 value: -12.181467961565222
step: 21 value: -18.82188746216514
step: 22 value: -18.777447050650593
step: 23 value: -11.278520483966087
step: 24 value: -19.821092937617262
step: 25 value: -18.827522928006285
step: 26 value: -19.82110850394644
step: 27 value:

So, for N=8 atoms, the best results come from using the BFGS method of optimization with taking only 2 steps on average and aqbout 1 second time cost. Then the second best would be the CG method taking on average 5 steps with about 3 second time cost. The other two methods do not produce reliable results, they take many iterations and take more time.

## For N=10 atoms

In [14]:
f1_N10_values = []
x1_N10_values = []
N_attempts = 20
N_atom = 10

start = timeit.default_timer()

print('CG minimization method\n')

for i in range(N_attempts+10):
    pos = init_pos(N_atom)
    res1N10 = minimize(total_energy, pos, method='CG', tol=1e-4)
    f1_N10_values.append(res1N10.fun)
    x1_N10_values.append(res1N10.x)
    print('step:', i, 'value:', res1N10.fun)
        
stop = timeit.default_timer()    

print('\nGlobal min:', min(f1_N10_values))

pos = get_pos_from_url(N=10)
print('Real ground state:', total_energy(pos))
print('\ntime elapsed:', stop-start, 'seconds')
print('found on average 11 steps')

CG minimization method

step: 0 value: -27.545206852022513
step: 1 value: -26.418356107320076
step: 2 value: -25.427039332175514
step: 3 value: -25.33781176856015
step: 4 value: -27.479738906350928
step: 5 value: -27.52233011284828
step: 6 value: -27.54520685180874
step: 7 value: -12.302927946866852
step: 8 value: -27.545206851721876
step: 9 value: -27.54520685214568
step: 10 value: -25.427039331853504
step: 11 value: -26.41835610707188
step: 12 value: -26.558010931944526
step: 13 value: -26.521741604076013
step: 14 value: -26.49049870121894
step: 15 value: -19.82148919203636
step: 16 value: -24.11337705086608
step: 17 value: -26.490498701512685
step: 18 value: -27.446828967458483
step: 19 value: -26.62331977151117
step: 20 value: -25.50400788986643
step: 21 value: -26.418356107509766
step: 22 value: -28.42253189331249
step: 23 value: -26.44288499174891
step: 24 value: -26.41835610721125
step: 25 value: -27.54520685208465
step: 26 value: -15.533060177912569
step: 27 value: -27.11490952

In [7]:
f2_N10_values = []
x2_N10_values = []

start = timeit.default_timer()

print('Nelder-Mead minimization method\n')

for j in range(N_attempts+75):
    pos = init_pos(N_atom)
    res2N10 = minimize(total_energy, pos, method='nelder-mead', tol=1e-4)
    f2_N10_values.append(res2N10.fun)
    x2_N10_values.append(res2N10.x)
    print('step:', j, 'value:', res2N10.fun)
        
stop = timeit.default_timer()    

print('\nGlobal min:', min(f2_N10_values))

pos = get_pos_from_url(N=10)
print('Real ground state:', total_energy(pos))
print('\ntime elapsed:', stop-start, 'seconds')
print('this method does not produce ground state after many iterations')

Nelder-Mead minimization method

step: 0 value: -9.647849541675292
step: 1 value: -8.95625714578124
step: 2 value: -7.2928776900313075
step: 3 value: -12.595141816783416
step: 4 value: -11.737994635605066
step: 5 value: -13.119943765184685
step: 6 value: -15.14272265606422
step: 7 value: -11.860415602326224
step: 8 value: -16.574760847738396
step: 9 value: -15.009800670159784
step: 10 value: -11.892376123991905
step: 11 value: -17.061482166129167
step: 12 value: -12.48015389190493
step: 13 value: -12.43250662321199
step: 14 value: -13.780996053180504
step: 15 value: -5.124772224054396
step: 16 value: -13.114656854848967
step: 17 value: -12.359959064346867
step: 18 value: -17.201395517529566
step: 19 value: -22.941300873475615
step: 20 value: -13.366436742473423
step: 21 value: -10.588834944612243
step: 22 value: -11.071191928467632
step: 23 value: -10.565174367657503
step: 24 value: -10.175279507715631
step: 25 value: -10.350203898732747
step: 26 value: -10.488067268825715
step: 27 val

In [8]:
f3_N10_values = []
x3_N10_values = []

start = timeit.default_timer()

print('BFGS minimization method\n')

for k in range(N_attempts):
    pos = init_pos(N_atom)
    res3N10 = minimize(total_energy, pos, method='BFGS', tol=1e-4)
    f3_N10_values.append(res3N10.fun)
    x3_N10_values.append(res3N10.x)
    print('step:', k, 'value:', res3N10.fun)
    
stop = timeit.default_timer()    

print('\nGlobal min:', min(f3_N10_values))

pos = get_pos_from_url(N=10)
print('Real ground state:', total_energy(pos))
print('\ntime elapsed:', stop-start, 'seconds')
print('found on average of 4 steps')

BFGS minimization method

step: 0 value: -15.53308085701753
step: 1 value: -26.483909045224863
step: 2 value: -26.483909045141516
step: 3 value: -15.533060058502853
step: 4 value: -27.114909528716737
step: 5 value: -27.479738906217545
step: 6 value: -25.407481840714524
step: 7 value: -22.1863607842824
step: 8 value: -27.205646971133845
step: 9 value: -25.288749213314688
step: 10 value: -27.479738906353393
step: 11 value: -18.82867154953845
step: 12 value: -26.558010932141325
step: 13 value: -26.442884991985252
step: 14 value: -27.446828967598982
step: 15 value: -26.490498701989942
step: 16 value: -26.490498701965297
step: 17 value: -26.558010932143848
step: 18 value: -27.19010250163104
step: 19 value: -28.42253189336559

Global min: -28.42253189336559
Real ground state: -28.422531893437565

time elapsed: 2.889773613000216 seconds
found on average of 4 steps


In [9]:
f4_N10_values = []
x4_N10_values = []

start = timeit.default_timer()

print('Powell minimization method\n')

for m in range(N_attempts+30):
    pos = init_pos(N_atom)
    res4N10 = minimize(total_energy, pos, method='powell', tol=1e-4)
    f4_N10_values.append(res4N10.fun)
    x4_N10_values.append(res4N10.x)
    print('step:', m, 'value:', res4N10.fun)
        
stop = timeit.default_timer()    

print('\nGlobal min:', min(f4_N10_values))

pos = get_pos_from_url(N=10)
print('Real ground state:', total_energy(pos))
print('\ntime elapsed:', stop-start, 'seconds')
print('this method does not produce ground state')

Powell minimization method

step: 0 value: -26.557824615016635
step: 1 value: -26.607746394058516
step: 2 value: -18.33766380976926
step: 3 value: -27.290381227125472
step: 4 value: -26.14599539794648
step: 5 value: -18.62206154474125
step: 6 value: -26.557944382511366
step: 7 value: -27.543735113021647
step: 8 value: -18.56758998344031
step: 9 value: -25.33219165113521
step: 10 value: -26.48879188961574
step: 11 value: -28.411339838120178
step: 12 value: -25.40160221549004
step: 13 value: -20.87806997680792
step: 14 value: -27.445118668456747
step: 15 value: -27.479476158998853
step: 16 value: -26.32349842776592
step: 17 value: -18.366608068104807
step: 18 value: -27.53976633835116
step: 19 value: -25.286403210286363
step: 20 value: -18.387656040714344
step: 21 value: -26.488276605927826
step: 22 value: -25.284603269574276
step: 23 value: -18.33931377529603
step: 24 value: -18.552958836738743
step: 25 value: -26.555678254429065
step: 26 value: -27.1109426197773
step: 27 value: -27.476

For N=10 atoms, BFGS was the most efficient method to find the ground state with an average of 4 steps and about 4 seconds time cost. CG also was able to succeed but an average of 11 steps but with about 7 seconds time cost. Powell and Nelder-Mead method took many iteration and did not produce the correct results.

## For N=13 atoms

In [15]:
f1_N13_values = []
x1_N13_values = []
N_attempts = 20
N_atom = 13

start = timeit.default_timer()

print('CG minimization method\n')

for i in range(N_attempts+15):
    pos = init_pos(N_atom)
    res1N13 = minimize(total_energy, pos, method='CG', tol=1e-4)
    f1_N13_values.append(res1N13.fun)
    x1_N13_values.append(res1N13.x)
    print('step:', i, 'value:', res1N13.fun)
        
stop = timeit.default_timer()    

print('\nGlobal min:', min(f1_N13_values))

pos = get_pos_from_url(N=13)
print('Real ground state:', total_energy(pos))
print('\ntime elapsed:', stop-start, 'seconds')
print('found on average of 18 steps')

CG minimization method

step: 0 value: -39.758643078295634
step: 1 value: -36.79938618376919
step: 2 value: -38.84210756907626
step: 3 value: -38.225712516506654
step: 4 value: -24.113360433694783
step: 5 value: -38.78036627829243
step: 6 value: -39.546690409254964
step: 7 value: -38.695205105100634
step: 8 value: -30.821917444222855
step: 9 value: -38.39775236129962
step: 10 value: -39.57188351093292
step: 11 value: -39.52963955733749
step: 12 value: -39.674897819623865
step: 13 value: -39.8382604040284
step: 14 value: -40.67017034682708
step: 15 value: -39.5370553369611
step: 16 value: -38.816897299535086
step: 17 value: -29.736381389908807
step: 18 value: -39.52963955725399
step: 19 value: -30.546203282200626
step: 20 value: -44.32680141950264
step: 21 value: -36.40344890759007
step: 22 value: -30.753650398709098
step: 23 value: -36.59333957206767
step: 24 value: -38.79795928011152
step: 25 value: -41.4445970020859
step: 26 value: -38.641965393434525
step: 27 value: -39.430715961191

In [16]:
f2_N13_values = []
x2_N13_values = []

start = timeit.default_timer()

print('BFGS minimization method\n')

for j in range(N_attempts):
    pos = init_pos(N_atom)
    res2N13 = minimize(total_energy, pos, method='BFGS', tol=1e-4)
    f2_N13_values.append(res2N13.fun)
    x2_N13_values.append(res2N13.x)
    print('step:', j, 'value:', res2N13.fun)

stop = timeit.default_timer()    

print('\nGlobal min:', min(f2_N13_values))

pos = get_pos_from_url(N=13)
print('Real ground state:', total_energy(pos))
print('\ntime elapsed:', stop-start, 'seconds')
print('found on average of 12 steps')

BFGS minimization method

step: 0 value: -26.442891548263713
step: 1 value: -39.68253362502172
step: 2 value: -40.01755319234057
step: 3 value: -39.56299952916942
step: 4 value: -27.522360003625202
step: 5 value: -30.821917312886217
step: 6 value: -44.32680141926046
step: 7 value: -37.96766369006804
step: 8 value: -39.692427035330844
step: 9 value: -30.753650399057328
step: 10 value: -38.397752361553025
step: 11 value: -39.44601062302311
step: 12 value: -38.424654429412115
step: 13 value: -30.88743005548278
step: 14 value: -38.431337219862606
step: 15 value: -39.56299952943277
step: 16 value: -35.21290537375318
step: 17 value: -40.61546780041217
step: 18 value: -38.526287674006234
step: 19 value: -26.49049997656788

Global min: -44.32680141926046
Real ground state: -44.32680141873467

time elapsed: 5.52728192900031 seconds
found on average of 12 steps


In [12]:
f3_N13_values = []
x3_N13_values = []

start = timeit.default_timer()

print('Nelder-Mead minimization method\n')

for k in range(N_attempts+50):
    pos = init_pos(N_atom)
    res3N13 = minimize(total_energy, pos, method='nelder-mead', tol=1e-4)
    f3_N13_values.append(res3N13.fun)
    x3_N13_values.append(res3N13.x)
    print('step:', k, 'value:', res3N13.fun)
        
stop = timeit.default_timer()    

print('\nGlobal min:', min(f3_N13_values))

pos = get_pos_from_url(N=13)
print('Real ground state:', total_energy(pos))
print('\ntime elapsed:', stop-start, 'seconds')
print('this method does not produce the proper ground state')

Nelder-Mead minimization method

step: 0 value: -19.551428273488447
step: 1 value: -18.796481048260187
step: 2 value: -18.891758590135105
step: 3 value: -13.557897153428918
step: 4 value: -19.98165476797073
step: 5 value: -15.5519458789152
step: 6 value: -20.270095095489317
step: 7 value: -17.466708857740112
step: 8 value: -19.464489077916042
step: 9 value: -16.741512147483324
step: 10 value: -20.181165931846024
step: 11 value: -24.22647678843686
step: 12 value: -17.493049703982923
step: 13 value: -13.698743642704114
step: 14 value: -15.294379564712575
step: 15 value: -20.219049481558077
step: 16 value: -16.55248761025672
step: 17 value: -16.39300850516084
step: 18 value: -23.496772462325843
step: 19 value: -22.776210277844033
step: 20 value: -26.263710107681593
step: 21 value: -18.423651376535457
step: 22 value: -15.680353198356594
step: 23 value: -16.311530848891277
step: 24 value: -15.729778547958212
step: 25 value: -19.807781225040422
step: 26 value: -20.456039697176422
step: 27 va

In [13]:
f4_N13_values = []
x4_N13_values = []

start = timeit.default_timer()

print('Powell minimization method\n')

for m in range(N_attempts+30):
    pos = init_pos(N_atom)
    res4N13 = minimize(total_energy, pos, method='powell', tol=1e-4)
    f4_N13_values.append(res4N13.fun)
    x4_N13_values.append(res4N13.x)
    print('step:', m, 'value:', res4N13.fun)
        
stop = timeit.default_timer()    

print('\nGlobal min:', min(f4_N13_values))

pos = get_pos_from_url(N=13)
print('Real ground state:', total_energy(pos))
print('\ntime elapsed:', stop-start, 'seconds')
print('this method does not produce proper results')

Powell minimization method

step: 0 value: -38.7825978960921
step: 1 value: -21.30112715329368
step: 2 value: -39.48755240783252
step: 3 value: -29.462082162281504
step: 4 value: -38.3269189810745
step: 5 value: -38.588758133134945
step: 6 value: -39.440514330218065
step: 7 value: -36.39939724410562
step: 8 value: -36.898470081217624
step: 9 value: -36.27753657337076
step: 10 value: -28.06183595434426
step: 11 value: -26.05347297970065
step: 12 value: -29.295614168263125
step: 13 value: -39.76798923247281
step: 14 value: -28.056675129615943
step: 15 value: -31.775657970429762
step: 16 value: -27.023135072105266
step: 17 value: -36.584905704635155
step: 18 value: -39.64430259474931
step: 19 value: -37.630014450875535
step: 20 value: -37.71089229758205
step: 21 value: -27.98820752863346
step: 22 value: -28.10531448758368
step: 23 value: -39.229521308721075
step: 24 value: -36.669140398286025
step: 25 value: -35.12964245866894
step: 26 value: -39.621922121801475
step: 27 value: -37.226873

For N=13 atoms, BFGS was the most efficient producing the correct results after an average of 12 steps and about 5 seconds. CG was also able to produce correct results but took an average of 18 steps and about 19 seconds. The powell and nelder-mead method did not produce proper results after many iterations.

## Conclusion:

The BFGS method is the most efficient optimization method using scipy's minimization functions. It has the least time cost and the least amount of steps on average to get the correct ground state.

The CG method was also successful at finding the ground state but took more only a little bit more steps on average but sometimes a lot more extra time cost.

The Nelder-Mead and Powell methods were unsuccessful at producing the proper results even after many iterations and took a long time to no succeed.